2023/09/26

# OpenBCI Python版GUI

💻使い方
- `pip install -U setuptools numpy scipy brainflow pygame`
- あとは上から実行していくだけ．
  - `Board()` の `port` 引数だけ変える必要あるかも（詳細は今後の更新で）

## Why オレオレ GUI?
- 😎CS学徒にとって実行・実験管理が簡単
  - 脳波時系列データはGUI実行中も実行後も，好きなタイミングで `board.get_board_data()` で取得し，解析できる．
  - 刺激呈示と脳波の時刻合わせをプログラムで厳密に管理できる．
- 🤩必要最低限のコード
  - カスタマイズが容易
  - 脳波の原理からの理解に役立つ

In [2]:
import numpy as np
from scipy import signal
import brainflow, pygame
from pygame.locals import *
colors = [(255, 255, 255), (0,0,255), (0,255,255), (0,255,0), (255,255,0), (255,0,0), (255,0,255), (255,255,255)]

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def normalize(X, sfreq=250, fmin=5, fmax=40):
    X = signal.lfilter(*signal.iirnotch(50, 4, sfreq), X, axis=1)
    X = signal.lfilter(*signal.butter(5, np.array([fmin, fmax])/(sfreq/2), btype="bandpass"), X, axis=1)
    return X

In [4]:
class Board(brainflow.BoardShim):
    def __init__(self, port="COM3", sfreq=250):
        params = brainflow.BrainFlowInputParams()
        params.serial_port = port
        super().__init__(brainflow.BoardIds.CYTON_BOARD, params)

        self.prepare_session()
        for ch in range(1, 9):
            self.config_board(f"x{ch}030110X")
            print(end=".")
        print("ready")
        self.start_stream()
        
        self.data_tot = np.zeros((8, 1000))
        self.sfreq = sfreq

    def get_data(self, trange=3):
        N = trange * self.sfreq
        data_new = self.get_board_data()[1:9]
        self.data_tot = np.concatenate([self.data_tot, data_new], 1)
        X = normalize(self.data_tot[:, -2*N:], fmin=5)[:, -N:]
        return X
    
    def switch_stream(self):
        if self.is_streaming:
            self.stop_stream()
        else:
            self.start_stream()

    def start_stream(self, num_samples: int = 1800 * 250, streamer_params: str = None) -> None:
        self.is_streaming = True
        return super().start_stream(num_samples, streamer_params)
    
    def stop_stream(self) -> None:
        self.is_streaming = False
        return super().stop_stream()

In [47]:
class App:
    def __init__(self, board, width=640, height=480, fps=20):
        self.W = width
        self.H = height
        self.fps = fps
        self.board = board
        pygame.init()
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont("Times New Roman", 20)
        
    def init_screen(self):
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    self.board.switch_stream()
            if event.type == pygame.QUIT:
                pygame.quit()
                return pygame.QUIT
        self.clock.tick(self.fps)
        self.screen.fill(0)

    def draw_waves(self, data, left=0, top=0, right=640, bottom=480, trange=3, maxuV=200):
        W = right - left
        H = bottom - top
        h = H // 16 # 振幅の半分のピクセル数
        x = np.linspace(left, right, data.shape[1])
        Y = h + data.clip(-maxuV, maxuV) * h / maxuV
        for i, y in enumerate(Y):
            topi = top + 2 * h * i
            pygame.draw.lines(self.screen, colors[i], False, np.c_[x, y+topi])
            pygame.draw.line(self.screen, "white", (left, topi), (right, topi))
        pygame.draw.line(self.screen, "white", (left, bottom), (right, bottom))

        for x in np.linspace(right, left, trange+1).astype(int):
            pygame.draw.line(self.screen, "gray", (x, top), (x, bottom))

    def draw_spec(self, data, left=0, top=0, right=640, bottom=480, lim=100):
        freq, Vs = signal.welch(data, self.board.sfreq, scaling="spectrum")
        As_log = np.log(Vs[:, :63] * data.shape[1] * 1e-3)
        Y = (top - bottom) * As_log.clip(0, lim) / lim + bottom
        x = np.linspace(left, right, Y.shape[1])
        for i, y in enumerate(Y):
            pygame.draw.lines(self.screen, colors[i], False, np.c_[x, y])
        for freq in range(0, 61, 10):
            x = left + (right-left) * freq / 60
            pygame.draw.line(self.screen, "gray", (x, top), (x, bottom))
        pygame.draw.line(self.screen, "gray", (left, top), (right, top))
        pygame.draw.line(self.screen, "gray", (left, bottom), (right, bottom))

    def main(self, trange=3):
        for frame in range(9999):
            if self.init_screen() == pygame.QUIT:
                break
            data = self.board.get_data(trange)
            self.draw_waves(data, 0, 0, self.W//2, self.H, trange)
            self.draw_spec(data, self.W//2, 0, self.W, self.H//2, lim=20)
            pygame.display.update()

In [7]:
board = Board(port="COM3")  # USBドングル挿してるポートを指定．Ubuntuなら port="/dev/ttyUSB0" など

........ready


In [48]:
app = App(board, 960, 640)
app.main()

In [165]:
board.release_session()